data modification based on requirements

In [131]:
import pandas as pd

def create_simplified_csv(input_csv_path, output_csv_path):
    # Load original data
    df = pd.read_csv(input_csv_path)

    # Calculate TotalBathrooms
    df['TotalBathrooms'] = (
        df['FullBath'] + 0.5 * df['HalfBath'] +
        df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath']
    )

    # Select required columns
    simplified_df = df[['GrLivArea', 'BedroomAbvGr', 'TotalBathrooms']]

    # Drop rows with missing values in these columns (optional)
    simplified_df = simplified_df.dropna()

    # Save to new CSV
    simplified_df.to_csv(output_csv_path, index=False)
    print(f"Simplified CSV saved to {output_csv_path}")

# Example usage
input_path = '/content/test.csv'    # Change this to your CSV path
output_path = 'simplified_data.csv'      # Desired output path

create_simplified_csv(input_path, output_path)


Simplified CSV saved to simplified_data.csv


smoting

In [110]:
import pandas as pd
import numpy as np

def augment_data(X, y, factor=10, noise_level=0.05, random_state=42):
    """
    Augment data by creating synthetic samples by adding Gaussian noise to numeric features.
    Args:
        X (pd.DataFrame): Features dataframe.
        y (pd.Series): Target values.
        factor (int): How many times to increase dataset size (including original).
        noise_level (float): Relative noise level (fraction of feature value).
        random_state (int): For reproducibility.
    Returns:
        X_aug (pd.DataFrame), y_aug (pd.Series)
    """
    np.random.seed(random_state)
    n_samples = X.shape[0]
    n_augmented = n_samples * (factor - 1)

    synthetic_rows = []
    synthetic_targets = []

    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

    for _ in range(n_augmented):
        idx = np.random.randint(0, n_samples)
        sample = X.iloc[idx].copy()
        target = y.iloc[idx]

        # Add noise to numeric features only
        for col in numeric_cols:
            val = sample[col]
            if pd.notnull(val):
                noise = np.random.normal(loc=0.0, scale=noise_level * max(abs(val), 1))
                sample[col] = max(val + noise, 0)  # ensure no negative values

        synthetic_rows.append(sample)
        synthetic_targets.append(target)

    X_synth = pd.DataFrame(synthetic_rows, columns=X.columns)
    y_synth = pd.Series(synthetic_targets)

    # Combine original + synthetic
    X_aug = pd.concat([X, X_synth], ignore_index=True)
    y_aug = pd.concat([y, y_synth], ignore_index=True)

    return X_aug, y_aug


def create_and_augment_csv(input_csv_path, output_csv_path):
    # Load original data
    df = pd.read_csv(input_csv_path)

    # Calculate TotalBathrooms
    df['TotalBathrooms'] = (
        df['FullBath'] + 0.5 * df['HalfBath'] +
        df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath']
    )

    # Select required columns
    simplified_df = df[['GrLivArea', 'BedroomAbvGr', 'TotalBathrooms', 'SalePrice']]

    # Drop rows with missing values
    simplified_df = simplified_df.dropna()

    # Separate features and target
    X = simplified_df.drop(columns=['SalePrice'])
    y = simplified_df['SalePrice']

    # Augment data 10x with noise_level 1%
    X_aug, y_aug = augment_data(X, y, factor=3, noise_level=0.0001)

    # Combine back to DataFrame
    augmented_df = X_aug.copy()
    augmented_df['SalePrice'] = y_aug

    # Save to CSV
    augmented_df.to_csv(output_csv_path, index=False)
    print(f"Augmented CSV saved to {output_csv_path}")
    print(f"Original dataset size: {simplified_df.shape[0]}, Augmented dataset size: {augmented_df.shape[0]}")

# Example usage
input_path = '/content/train.csv'  # Change to your CSV path
output_path = 'augmented_simplified_data.csv'

create_and_augment_csv(input_path, output_path)


Augmented CSV saved to augmented_simplified_data.csv
Original dataset size: 1460, Augmented dataset size: 4380


training section

try 1

In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Load simplified dataset CSV
data = pd.read_csv('/content/augmented_simplified_data.csv')  # Change path if needed

# Features and target
X = data[['GrLivArea', 'BedroomAbvGr', 'TotalBathrooms']]
y = data['SalePrice']

# Initialize Linear Regression model
model = LinearRegression()

# Cross-validation (5-fold) RMSE
neg_mse_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-neg_mse_scores)
print(f'Cross-Validated RMSE: {rmse_scores.mean():.2f}')

# Train final model on full dataset
model.fit(X, y)

# Predict on a new sample
new_house = pd.DataFrame([{
    'GrLivArea': 1500,
    'BedroomAbvGr': 3,
    'TotalBathrooms': 2.5  # example: 2 full baths + 1 half bath
}])

predicted_price = model.predict(new_house)[0]
print(f'Predicted price for new house: ${predicted_price:,.2f}')


Cross-Validated RMSE: 49801.06
Predicted price for new house: $185,023.92


try 2

In [36]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# Load the simplified dataset with pre-engineered TotalBathrooms
data = pd.read_csv('/content/augmented_simplified_data.csv')  # path to your CSV

# Example feature engineering: BathroomDensity = TotalBathrooms per 1000 sqft
data['BathroomDensity'] = data['TotalBathrooms'] / (data['GrLivArea'] / 1000)

# Features and target
X = data[['GrLivArea', 'BedroomAbvGr', 'TotalBathrooms', 'BathroomDensity']]
y = data['SalePrice']

# Drop missing values if any
Xy = pd.concat([X, y], axis=1).dropna()
X = Xy.drop(columns=['SalePrice'])
y = Xy['SalePrice']

# Initialize model
model = LinearRegression()

# Cross-validation RMSE
neg_mse_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-neg_mse_scores)
print(f'Cross-Validated RMSE: {rmse_scores.mean():.2f}')

# Train final model
model.fit(X, y)

# Predict on a new sample with the new feature
new_house = pd.DataFrame([{
    'GrLivArea': 1500,
    'BedroomAbvGr': 3,
    'TotalBathrooms': 2.5,
    'BathroomDensity': 2.5 / (1500 / 1000)  # = 2.5 / 1.5 = 1.6667
}])

predicted_price = model.predict(new_house)[0]
print(f'Predicted price for new house: ${predicted_price:,.2f}')


Cross-Validated RMSE: 46457.25
Predicted price for new house: $186,562.76


parameter tuning

In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Load the simplified dataset with pre-engineered TotalBathrooms
data = pd.read_csv('/content/augmented_simplified_data.csv')  # path to your CSV

# Feature engineering: BathroomDensity = TotalBathrooms per 1000 sqft
data['BathroomDensity'] = data['TotalBathrooms'] / (data['GrLivArea'] / 1000)

# Features and target
X = data[['GrLivArea', 'BedroomAbvGr', 'TotalBathrooms', 'BathroomDensity']]
y = data['SalePrice']

# Drop missing values if any
Xy = pd.concat([X, y], axis=1).dropna()
X = Xy.drop(columns=['SalePrice'])
y = Xy['SalePrice']

# Define Ridge regression model
ridge = Ridge()

# Parameter grid for alpha
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Setup GridSearchCV
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

print(f"Best alpha: {grid_search.best_params_['alpha']}")

# Best model
best_model = grid_search.best_estimator_

# Cross-validated RMSE for best model
rmse_scores = (-grid_search.cv_results_['mean_test_score'])**0.5
print(f"Grid Search RMSEs: {rmse_scores}")

print(f'Cross-Validated RMSE of best model: {rmse_scores[grid_search.best_index_]:.2f}')

# Train best model on full data
best_model.fit(X, y)

# Predict on new sample
new_house = pd.DataFrame([{
    'GrLivArea': 1500,
    'BedroomAbvGr': 3,
    'TotalBathrooms': 2.5,
    'BathroomDensity': 2.5 / (1500 / 1000)
}])

predicted_price = best_model.predict(new_house)[0]
print(f'Predicted price for new house: ${predicted_price:,.2f}')


Best alpha: 0.1
Grid Search RMSEs: [46550.41186215 46550.40864544 46550.38071424 46550.50740727
 46579.20811409 47025.1405305 ]
Cross-Validated RMSE of best model: 46550.38
Predicted price for new house: $186,556.24


multi model try

In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv('/content/augmented_simplified_data.csv')

# Feature engineering
data['BathroomDensity'] = data['TotalBathrooms'] / (data['GrLivArea'] / 1000)

# Drop missing values
data = data.dropna(subset=['GrLivArea', 'BedroomAbvGr', 'TotalBathrooms', 'BathroomDensity', 'SalePrice'])

# Define features and target
X1 = data[['GrLivArea', 'BedroomAbvGr']]
X2 = data[['TotalBathrooms', 'BathroomDensity']]
y = data['SalePrice']

# Train-test split 80/20
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    X1, X2, y, test_size=0.2, random_state=42)

# Initialize models
model1 = Ridge(alpha=1.0)
model2 = Ridge(alpha=1.0)

# Train models on training data
model1.fit(X1_train, y_train)
model2.fit(X2_train, y_train)

# Predict on test data
preds1 = model1.predict(X1_test)
preds2 = model2.predict(X2_test)

# Combined predictions by averaging
combined_preds = (preds1 + preds2) / 2

# Calculate RMSE for test data
rmse1 = mean_squared_error(y_test, preds1)
rmse2 = mean_squared_error(y_test, preds2)
rmse_combined = mean_squared_error(y_test, combined_preds)

print(f"Test RMSE Model1 (GrLivArea, BedroomAbvGr): {rmse1:.2f}")
print(f"Test RMSE Model2 (TotalBathrooms, BathroomDensity): {rmse2:.2f}")
print(f"Test RMSE Combined Model (average): {rmse_combined:.2f}")

# Predict on a new sample
new_house = pd.DataFrame([{
    'GrLivArea': 1500,
    'BedroomAbvGr': 3,
    'TotalBathrooms': 2.5,
    'BathroomDensity': 2.5 / (1500 / 1000)
}])

pred1 = model1.predict(new_house[['GrLivArea', 'BedroomAbvGr']])[0]
pred2 = model2.predict(new_house[['TotalBathrooms', 'BathroomDensity']])[0]
final_pred = (pred1 + pred2) / 2

print(f"\nPrediction on new house:")
print(f"Model1 prediction: ${pred1:,.2f}")
print(f"Model2 prediction: ${pred2:,.2f}")
print(f"Final averaged prediction: ${final_pred:,.2f}")


Test RMSE Model1 (GrLivArea, BedroomAbvGr): 2165391366.67
Test RMSE Model2 (TotalBathrooms, BathroomDensity): 2378146336.36
Test RMSE Combined Model (average): 2070130902.37

Prediction on new house:
Model1 prediction: $175,311.52
Model2 prediction: $193,876.63
Final averaged prediction: $184,594.08


new try

In [69]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the CSV file (replace with your actual file path)
csv_file_path = "/content/augmented_simplified_data.csv"
df = pd.read_csv(csv_file_path)

# Define features and target
X = df[["GrLivArea", "BedroomAbvGr", "TotalBathrooms"]]
y = df["SalePrice"]

# Split data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Optional: print model coefficients
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)


Mean Squared Error: 1881513831.2312655
Coefficients: [    94.37963102 -21825.47571341  30158.57399427]
Intercept: 33455.71940686804


parameter tuning

In [126]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Load data
csv_file_path = "/content/augmented_simplified_data.csv"
df = pd.read_csv(csv_file_path)

X = df[["GrLivArea", "BedroomAbvGr", "TotalBathrooms"]]
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Ridge model
ridge = Ridge()

# Define hyperparameter grid for alpha (regularization strength)
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 50.0, 100.0, 200.0]
}

# Set up GridSearchCV to find best alpha
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Best model after tuning
best_ridge = grid_search.best_estimator_

# Predict and evaluate on test data
y_pred = best_ridge.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("Best alpha:", grid_search.best_params_['alpha'])
print("Mean Squared Error (test):", mse)
print("Coefficients:", best_ridge.coef_)
print("Intercept:", best_ridge.intercept_)

# R^2 score as accuracy percentage
r2 = best_ridge.score(X_test, y_test)
print(f"R^2 Score: {r2:.4f} ({r2*100:.2f}%)")


Best alpha: 0.1
Mean Squared Error (test): 2382486486.940036
Coefficients: [   100.70530626 -23227.16147906  30096.81061372]
Intercept: 28630.750532433536
R^2 Score: 0.6108 (61.08%)


In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

# Load data
csv_file_path = "/content/augmented_simplified_data.csv"
df = pd.read_csv(csv_file_path)

X = df[["GrLivArea", "BedroomAbvGr", "TotalBathrooms"]]
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define pipeline steps:
# 1. Scale features
# 2. Generate polynomial features (degree 2)
# 3. ElasticNet regression model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('elasticnet', ElasticNet(max_iter=10000))
])

# Hyperparameter grid to tune alpha (regularization) and l1_ratio (L1 vs L2 mixing)
param_grid = {
    'elasticnet__alpha': [0.01, 0.1, 1, 10, 100],
    'elasticnet__l1_ratio': [0.1, 0.5, 0.7, 0.9, 1]  # 1 = Lasso, 0 = Ridge
}

# Setup GridSearchCV with 5-fold CV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predict and evaluate on test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = best_model.score(X_test, y_test)

print("Best hyperparameters:", grid_search.best_params_)
print(f"Test Mean Squared Error: {mse:.2f}")
print(f"Test R^2 Score: {r2:.4f} ({r2*100:.2f}%)")


Best hyperparameters: {'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.7}
Test Mean Squared Error: 1891961572.06
Test R^2 Score: 0.6638 (66.38%)


polynomial featuring


In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Load data
csv_file_path = "/content/augmented_simplified_data.csv"
df = pd.read_csv(csv_file_path)

X = df[["GrLivArea", "BedroomAbvGr", "TotalBathrooms"]]
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline: scaling -> polynomial features -> ridge regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=4, include_bias=False)),
    ('ridge', Ridge(alpha=1.0))
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = pipeline.score(X_test, y_test)

print("Mean Squared Error:", mse)
print(f"R^2 Score: {r2:.4f} ({r2*100:.2f}%)")


Mean Squared Error: 1540399180.5147827
R^2 Score: 0.7262 (72.62%)


best params

In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Load data
csv_file_path = "/content/augmented_simplified_data.csv"
df = pd.read_csv(csv_file_path)

# Select features and target
X = df[["GrLivArea", "BedroomAbvGr", "TotalBathrooms"]]
y = df["SalePrice"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=4, include_bias=False)),
    ('ridge', Ridge())
])

# Hyperparameter grid for tuning alpha
param_grid = {
    'ridge__alpha': [0.01, 0.1, 1, 10, 100]
}

# Grid search with 5-fold CV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Train model and find best hyperparameters
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = best_model.score(X_test, y_test)

print("Best alpha:", grid_search.best_params_['ridge__alpha'])
print("Test Mean Squared Error:", mse)
print(f"Test R^2 Score: {r2:.4f} ({r2*100:.2f}%)")


Best alpha: 0.01
Test Mean Squared Error: 1540436914.853093
Test R^2 Score: 0.7262 (72.62%)


In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Load data
csv_file_path = "/content/augmented_simplified_data.csv"
df = pd.read_csv(csv_file_path)

# Features and target
X = df[["GrLivArea", "BedroomAbvGr", "TotalBathrooms"]]
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline with locked alpha
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=4, include_bias=False)),
    ('ridge', Ridge(alpha=0.01))
])

# Train model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = pipeline.score(X_test, y_test)

print("Locked alpha: 0.01")
print("Test Mean Squared Error:", mse)
print(f"Test R^2 Score: {r2:.4f} ({r2*100:.2f}%)")


Locked alpha: 0.01
Test Mean Squared Error: 1637486086.594804
Test R^2 Score: 0.7325 (73.25%)


In [127]:
import joblib

# After training the model (e.g., `model`), and scaler `scaler`

# Save scaler and model separately or as a pipeline
# Here we save them together as a pipeline for convenience

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', scaler),
    ('ridge', model)
])

# Save the pipeline to disk
joblib.dump(pipeline, 'ridge_poly_model.pkl')
print("Model pipeline saved to ridge_poly_model.pkl")


Model pipeline saved to ridge_poly_model.pkl


test section

In [135]:
import joblib
import pandas as pd

# Load the saved pipeline
pipeline = joblib.load('ridge_poly_model.pkl')

# Example manual input (replace values as needed)
# Input features must match original features before polynomial expansion

input_data = {
    "GrLivArea": [1500],
    "BedroomAbvGr": [3],
    "TotalBathrooms": [2.5]
}

# Create DataFrame
input_df = pd.DataFrame(input_data)

# IMPORTANT: We must create the same polynomial features manually as in training

def create_poly_features(X, degree=4):
    import numpy as np
    from itertools import combinations_with_replacement

    X_poly = pd.DataFrame(index=X.index)
    feature_names = X.columns.tolist()

    # Degree 1
    for col in feature_names:
        X_poly[col] = X[col]

    # Degree 2 to degree 4
    for deg in range(2, degree+1):
        combos = combinations_with_replacement(feature_names, deg)
        for combo in combos:
            col_name = "_".join(combo)
            product = np.ones(len(X))
            for c in combo:
                product *= X[c]
            X_poly[col_name] = product
    return X_poly

# Create polynomial features for input
input_poly = create_poly_features(input_df, degree=4)

# Predict using loaded pipeline
predicted_price = pipeline.predict(input_poly)

print(f"Predicted SalePrice: {predicted_price[0]:.2f}")


Predicted SalePrice: 183329.61


moduling

In [136]:
import joblib
import pandas as pd
import numpy as np
from itertools import combinations_with_replacement

class HousePricePredictor:
    def __init__(self, model_path='/content/linear_house.pkl', degree=4):
        # Load saved pipeline (scaler + model)
        self.pipeline = joblib.load(model_path)
        self.degree = degree

    def create_poly_features(self, X):
        feature_names = X.columns.tolist()
        X_poly = pd.DataFrame(index=X.index)

        # Degree 1 (original features)
        for col in feature_names:
            X_poly[col] = X[col]

        # Higher degrees
        for deg in range(2, self.degree + 1):
            combos = combinations_with_replacement(feature_names, deg)
            for combo in combos:
                col_name = "_".join(combo)
                product = np.ones(len(X))
                for c in combo:
                    product *= X[c]
                X_poly[col_name] = product

        return X_poly

    def predict(self, GrLivArea, BedroomAbvGr, TotalBathrooms):
        input_data = {
            "GrLivArea": [GrLivArea],
            "BedroomAbvGr": [BedroomAbvGr],
            "TotalBathrooms": [TotalBathrooms]
        }
        input_df = pd.DataFrame(input_data)
        input_poly = self.create_poly_features(input_df)
        predicted_price = self.pipeline.predict(input_poly)
        return predicted_price[0]


In [137]:
predictor = HousePricePredictor()

price = predictor.predict(1280, 2, 2.0)
print(f"Predicted SalePrice: {price:.2f}")


Predicted SalePrice: 165029.36
